## Toutes les catégories avec même taux de fraude --> taux de fraude ajusté proche de la réalité 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Distribution souhaitée pour les fraudes
FRAUD_DIST = {
    "Travel": 0.40,
    "Retail": 0.20,
    "Entertainment": 0.15,
    "Restaurant": 0.10,
    "Gas": 0.07,
    "Grocery": 0.05,
    "Healthcare": 0.02,
    "Education": 0.01
}


In [29]:

input_file = "transactions_clean.csv"
output_file = "transactions_rebalanced_2.csv"
chunksize = 500_000  # ajustable selon RAM

fraud_cats = list(FRAUD_DIST.keys())
fraud_w = list(FRAUD_DIST.values())

first_chunk = True  # pour écriture progressive

# --------------------------
# 2️⃣ Traitement chunk par chunk
# --------------------------
for chunk in pd.read_csv(input_file, chunksize=chunksize):

    # Assurer que is_fraud est booléen
    if chunk["is_fraud"].dtype != "bool":
        chunk["is_fraud"] = chunk["is_fraud"].astype(bool)

    # Séparer les dataframes
    df_fraud = chunk[chunk["is_fraud"] == True].copy()
    df_legit = chunk[chunk["is_fraud"] == False].copy()

    # --------------------------
    # 3️⃣ Rééchantillonner df_fraud selon FRAUD_DIST
    # --------------------------
    df_fraud_sampled = pd.DataFrame()

    for cat, prop in FRAUD_DIST.items():
        subset = df_fraud[df_fraud["merchant_category"] == cat]
        n_samples = int(len(df_fraud) * prop)  # nombre de lignes à tirer
        if len(subset) > 0:
            n_samples = min(n_samples, len(subset))  
            sampled = subset.sample(n=n_samples, replace=False)
            df_fraud_sampled = pd.concat([df_fraud_sampled, sampled], axis=0)


    # --------------------------
    # 4️⃣ Concaténer df_fraud_sampled + df_legit
    # --------------------------
    final_chunk = pd.concat([df_fraud_sampled, df_legit], axis=0)

    # --------------------------
    # 5️⃣ Écriture progressive
    # --------------------------
    if first_chunk:
        final_chunk.to_csv(output_file, index=False, mode="w")
        first_chunk = False
    else:
        final_chunk.to_csv(output_file, index=False, mode="a", header=False)

    print(f"✅ Chunk traité : {len(chunk)} lignes (fraudes rééchantillonnées)")

print("✅✅ Fichier final transactions_rebalanced.csv généré avec la distribution correcte pour les fraudes !")


✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 500000 lignes (fraudes rééchantillonnées)
✅ Chunk traité : 483766 lignes (fraudes rééchantillonnées)
✅✅ Fichier final transactions_rebalanced.csv généré avec la distribution correcte pour les fraudes !


In [3]:
from collections import Counter

counter = Counter()

for chunk in pd.read_csv(
    "transactions_rebalanced_2.csv",
    usecols=["country", "is_fraud", "merchant_category"],
    chunksize=300_000
):
    grouped = (
        chunk
        .groupby(["country", "is_fraud", "merchant_category"])
        .size()
    )
    counter.update(grouped.to_dict())

In [4]:
df_counts = (
    pd.DataFrame.from_dict(counter, orient="index", columns=["count"])
    .reset_index()
)

df_counts[["country", "is_fraud", "merchant_category"]] = pd.DataFrame(
    df_counts["index"].tolist(),
    index=df_counts.index
)

df_counts = df_counts.drop(columns="index")


In [5]:
df_counts["distribution"] = (
    (df_counts["count"]
    / df_counts.groupby(["country", "is_fraud"])["count"].transform("sum"))*100
)

In [6]:
pivot = df_counts.pivot_table(
    index="merchant_category",
    columns="is_fraud",
    values="distribution"
)

pivot

is_fraud,False,True
merchant_category,,
Education,12.477946,1.559657
Entertainment,12.509845,19.997853
Gas,12.500017,11.210540
Grocery,12.473410,7.951449
Healthcare,12.524003,3.191814
Restaurant,12.509679,16.026061
Retail,12.510897,19.917513
Travel,12.494203,20.145112
